In [1]:
import torch

from torch.utils.data import DataLoader

In [2]:
from lstm_scratch import SimpleLSTM
from helpers import CVFConfigForAnalysisDataset

In [3]:
model_name = "lstm_trained_at_2025_04_07_18_35"

graph_name = "star_graph_n15"

In [4]:
device = "cuda" 

In [5]:
# Model class must be defined somewhere
model = torch.load(f"trained_models/{model_name}.pt", weights_only=False)
model.eval()

SimpleLSTM(
  (lstm): LSTM(3, 32, batch_first=True)
  (h2o): Linear(in_features=32, out_features=1, bias=True)
)

In [6]:
dataset = CVFConfigForAnalysisDataset(device, graph_name)

Total configs: 245,760.


In [7]:
with open(f"ml_predictions/{model_name}__rank_effect_ml_pred__{graph_name}.txt", "w") as f:
    with torch.no_grad():
        test_dataloader = DataLoader(dataset, batch_size=1)

        count = 0
        for batch in test_dataloader:
            for i in range(len(batch[0])):
                frm_idx = batch[1][i].item()
                frm_rank = model(batch[0][i].unsqueeze(0))
                for (
                    position,
                    to_indx,
                ) in dataset.cvf_analysis.possible_perturbed_state_frm(frm_idx):
                    to = dataset[to_indx]
                    to_rank = model(to[0].unsqueeze(0))
                    f.write("%s" % (frm_rank - to_rank).item())
                    f.write(",")